In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

In [2]:
from dotenv import load_dotenv
import os
import sys
import urllib.request
import urllib.parse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
import time
import csv
import json
import requests
from datetime import datetime

In [ ]:
# 서비스 설정
service = Service(port=9999)
# 크롬 인터넷 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
options.add_argument('window-size=1380,900')
# 드라이버 생성
driver = webdriver.Chrome(service=service, options=options)
# WebDriverWait 초기화
wait = WebDriverWait(driver, 10)

# 결과 저장할 리스트
all_restaurant_data = []

def create_directory(keyword):
    """이미지 저장용 디렉토리 생성"""
    base_dir = f"{keyword}_restaurant_data"
    images_dir = os.path.join(base_dir, "images")
    os.makedirs(images_dir, exist_ok=True)
    return base_dir, images_dir

def download_image(url, filename, images_dir):
    """이미지 다운로드"""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            filepath = os.path.join(images_dir, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            return filepath
        return None
    except Exception as e:
        print(f"이미지 다운로드 실패 ({filename}): {e}")
        return None

def get_restaurant_list():
    """왼쪽 패널에서 가게 리스트 수집"""
    print("가게 리스트 수집 중...")
    
    # 스크롤하여 모든 가게 로드
    try:
        scroll_container = driver.find_element(By.CSS_SELECTOR, "div.Ryr1F")
        print("스크롤 컨테이너 찾음")
        
        previous_count = 0
        max_attempts = 15
        
        for attempt in range(max_attempts):
            restaurant_elements = driver.find_elements(By.CSS_SELECTOR, "li.UEzoS")
            current_count = len(restaurant_elements)
            
            print(f"현재 로드된 가게 수: {current_count}")
            
            if current_count == previous_count:
                if attempt >= 3:
                    print("더 이상 로드할 가게가 없음")
                    break
            else:
                previous_count = current_count
            
            # 스크롤 실행
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            time.sleep(2)
        
        return restaurant_elements
        
    except Exception as e:
        print(f"가게 리스트 수집 실패: {e}")
        return []

def click_restaurant(restaurant_element, index):
    """가게 클릭하여 세부 정보 패널 열기"""
    try:
        # 가게 링크 찾기
        link_element = restaurant_element.find_element(By.CSS_SELECTOR, "a.place_bluelink")
        
        # 스크롤하여 요소가 보이도록 함
        driver.execute_script("arguments[0].scrollIntoView(true);", link_element)
        time.sleep(1)
        
        # 클릭
        ActionChains(driver).move_to_element(link_element).click().perform()
        print(f"가게 {index} 클릭 완료")
        
        # 세부 정보 패널 로딩 대기
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"가게 {index} 클릭 실패: {e}")
        return False

def switch_to_detail_iframe():
    """오른쪽 세부 정보 iframe으로 전환"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 세부 정보 iframe 찾기
        detail_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
        driver.switch_to.frame(detail_iframe)
        print("세부 정보 iframe으로 전환 완료")
        return True
        
    except Exception as e:
        print(f"세부 정보 iframe 전환 실패: {e}")
        return False

def get_home_info():
    """홈 탭에서 기본 정보 수집"""
    home_info = {}
    
    try:
        # 가게 이름
        name_element = driver.find_element(By.CSS_SELECTOR, "span.GHAhO")
        home_info['name'] = name_element.text.strip()
        print(f"가게명: {home_info['name']}")
        
        # 가게 분류
        category_element = driver.find_element(By.CSS_SELECTOR, "span.lnJFt")
        home_info['category'] = category_element.text.strip()
        print(f"분류: {home_info['category']}")
        
        # 가게 주소
        try:
            address_element = driver.find_element(By.CSS_SELECTOR, "span.LDgIH")
            home_info['address'] = address_element.text.strip()
            print(f"주소: {home_info['address']}")
        except:
            home_info['address'] = "주소 정보 없음"
        
        # 전화번호 (있다면)
        try:
            phone_element = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q")
            home_info['phone'] = phone_element.text.strip()
        except:
            home_info['phone'] = "전화번호 정보 없음"
        
        # 영업시간 (있다면)
        try:
            hours_element = driver.find_element(By.CSS_SELECTOR, "time.H3ua4")
            home_info['hours'] = hours_element.text.strip()
        except:
            home_info['hours'] = "영업시간 정보 없음"
            
    except Exception as e:
        print(f"홈 정보 수집 실패: {e}")
        
    return home_info

def find_and_click_menu_tab():
    """메뉴 탭을 정확하게 찾아서 클릭"""
    try:
        print("메뉴 탭 찾는 중...")
        
        # 1단계: 모든 탭 요소들 찾기
        tab_selectors = [
            "a._tab-menu",
            "div.YYh8o a",
            "div[class*='tab'] a",
            "nav a",
            "ul li a"
        ]
        
        all_tabs = []
        for selector in tab_selectors:
            try:
                tabs = driver.find_elements(By.CSS_SELECTOR, selector)
                if tabs:
                    all_tabs = tabs
                    print(f"탭 요소들 찾음: {len(tabs)}개")
                    break
            except:
                continue
        
        if not all_tabs:
            print("탭 요소를 찾을 수 없습니다")
            return False
        
        # 2단계: 각 탭의 텍스트와 URL 확인하여 메뉴 탭 찾기
        menu_tab = None
        for i, tab in enumerate(all_tabs):
            try:
                # 탭 텍스트 확인
                tab_text = tab.text.strip().lower()
                # href 속성 확인
                tab_href = tab.get_attribute("href") or ""
                # data-index 확인
                tab_index = tab.get_attribute("data-index")
                
                print(f"탭 {i+1}: 텍스트='{tab_text}', href='{tab_href}', data-index='{tab_index}'")
                
                # 메뉴 탭인지 판단 (여러 조건으로 확인)
                is_menu_tab = (
                    "menu" in tab_href.lower() or
                    "메뉴" in tab_text or
                    "menu" in tab_text or
                    (tab_index and tab_index in ["1", "2"]) and "메뉴" in tab_text
                )
                
                if is_menu_tab:
                    menu_tab = tab
                    print(f"메뉴 탭 발견: {tab_text}")
                    break
                    
            except Exception as e:
                print(f"탭 {i+1} 확인 중 오류: {e}")
                continue
        
        # 3단계: 메뉴 탭 클릭
        if menu_tab:
            try:
                # 스크롤하여 요소가 보이도록 함
                driver.execute_script("arguments[0].scrollIntoView(true);", menu_tab)
                time.sleep(1)
                
                # JavaScript로 클릭
                driver.execute_script("arguments[0].click();", menu_tab)
                time.sleep(3)
                print("메뉴 탭 클릭 완료")
                
                # 메뉴 콘텐츠가 로드되었는지 확인
                menu_content_selectors = [
                    "li.E2jtL",  # 메뉴 항목
                    "div.place_section",  # 메뉴 섹션
                    "[class*='menu']"
                ]
                
                for selector in menu_content_selectors:
                    try:
                        elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        if elements:
                            print(f"메뉴 콘텐츠 확인됨: {len(elements)}개 요소")
                            return True
                    except:
                        continue
                
                print("메뉴 콘텐츠를 찾을 수 없지만 탭 클릭은 완료")
                return True
                
            except Exception as e:
                print(f"메뉴 탭 클릭 실패: {e}")
                return False
        else:
            print("메뉴 탭을 찾을 수 없습니다")
            return False
            
    except Exception as e:
        print(f"메뉴 탭 찾기 실패: {e}")
        return False

def click_more_menu_button():
    """메뉴 더보기 버튼 클릭"""
    try:
        # 여러 더보기 버튼 셀렉터 시도
        more_button_selectors = [
            "a.fvwqf",
            "button.fvwqf",
            "a[class*='more']",
            "button[class*='more']",
            "//a[contains(text(), '더보기')]",
            "//button[contains(text(), '더보기')]"
        ]
        
        for selector in more_button_selectors:
            try:
                if selector.startswith("//"):
                    more_button = driver.find_element(By.XPATH, selector)
                else:
                    more_button = driver.find_element(By.CSS_SELECTOR, selector)
                
                if more_button.is_displayed() and more_button.is_enabled():
                    # 요소가 보이도록 스크롤
                    driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                    time.sleep(1)
                    
                    # JavaScript로 클릭
                    driver.execute_script("arguments[0].click();", more_button)
                    time.sleep(2)
                    print("메뉴 더보기 버튼 클릭 완료")
                    return True
            except:
                continue
                
        print("메뉴 더보기 버튼이 없거나 이미 모든 메뉴가 표시됨")
        return False
        
    except Exception as e:
        print(f"메뉴 더보기 버튼 처리 실패: {e}")
        return False

def get_menu_photos(images_dir, restaurant_name):
    """메뉴판 이미지 다운로드"""
    menu_photos = []
    
    try:
        # 메뉴판 이미지 섹션 찾기 (여러 셀렉터 시도)
        section_selectors = [
            "div.place_section",
            "div[class*='photo']",
            "div[class*='image']",
            "div[class*='menu']"
        ]
        
        menu_photo_section = None
        for selector in section_selectors:
            try:
                menu_photo_section = driver.find_element(By.CSS_SELECTOR, selector)
                break
            except:
                continue
        
        if not menu_photo_section:
            print("메뉴판 이미지 섹션을 찾을 수 없습니다")
            return menu_photos
        
        # 이미지 요소들 찾기 (여러 셀렉터 시도)
        img_selectors = ["img.K0PDV", "img", "img[alt*='메뉴']"]
        img_elements = []
        
        for img_selector in img_selectors:
            try:
                elements = menu_photo_section.find_elements(By.CSS_SELECTOR, img_selector)
                if elements:
                    img_elements = elements
                    break
            except:
                continue
        
        for i, img_element in enumerate(img_elements):
            try:
                img_url = img_element.get_attribute("src")
                alt_text = img_element.get_attribute("alt", "").lower()
                
                # 메뉴판 이미지인지 확인
                if img_url and ("menu" in alt_text or "메뉴" in alt_text or len(alt_text) == 0):
                    # 메뉴판 이미지 파일명 생성
                    img_filename = f"{restaurant_name}_menu_photo_{i+1}.jpg"
                    img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                    
                    # 이미지 다운로드
                    downloaded_path = download_image(img_url, img_filename, images_dir)
                    if downloaded_path:
                        menu_photos.append(downloaded_path)
                        print(f"메뉴판 이미지 {i+1} 다운로드 완료")
            except Exception as e:
                print(f"메뉴판 이미지 {i+1} 다운로드 실패: {e}")
                continue
                
        # 메뉴판 이미지 슬라이드 버튼 클릭해서 더 많은 이미지 로드
        try:
            # 여러 슬라이드 버튼 셀렉터 시도
            slide_selectors = ["a.ZCqf_", "button.ZCqf_", "[class*='next']", "[class*='slide']"]
            
            for slide_selector in slide_selectors:
                try:
                    next_button = menu_photo_section.find_element(By.CSS_SELECTOR, slide_selector)
                    if next_button.is_displayed() and next_button.is_enabled():
                        click_count = 0
                        max_clicks = 5  # 최대 5번 클릭
                        
                        while click_count < max_clicks:
                            try:
                                # JavaScript로 클릭
                                driver.execute_script("arguments[0].click();", next_button)
                                time.sleep(1)
                                click_count += 1
                                
                                # 새로운 이미지들 찾기
                                new_img_elements = menu_photo_section.find_elements(By.CSS_SELECTOR, img_selector)
                                for j, new_img_element in enumerate(new_img_elements[len(img_elements):]):
                                    try:
                                        img_url = new_img_element.get_attribute("src")
                                        if img_url:
                                            img_filename = f"{restaurant_name}_menu_photo_{len(img_elements)+j+1}.jpg"
                                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                                            
                                            downloaded_path = download_image(img_url, img_filename, images_dir)
                                            if downloaded_path:
                                                menu_photos.append(downloaded_path)
                                    except:
                                        continue
                            except:
                                break
                        break
                except:
                    continue
                    
        except Exception as e:
            print(f"메뉴판 이미지 슬라이드 처리 실패: {e}")
            
    except Exception as e:
        print(f"메뉴판 이미지 수집 실패: {e}")
    
    return menu_photos

def get_menu_info(images_dir, restaurant_name):
    """메뉴 탭에서 메뉴 정보 수집"""
    menu_list = []
    
    try:
        # 메뉴 더보기 버튼 클릭
        click_more_menu_button()
        
        # 메뉴 항목들 찾기 (수정된 셀렉터)
        menu_elements = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
        
        for i, menu_element in enumerate(menu_elements):
            menu_info = {}
            
            try:
                # 메뉴명 (수정된 셀렉터)
                name_element = menu_element.find_element(By.CSS_SELECTOR, "span.lPzHi")
                menu_info['name'] = name_element.text.strip()
                
                # 메뉴 가격 (수정된 셀렉터)
                try:
                    price_element = menu_element.find_element(By.CSS_SELECTOR, "div.GXS1X em")
                    menu_info['price'] = price_element.text.strip()
                except:
                    menu_info['price'] = "가격 정보 없음"
                
                # 메뉴 설명 (수정된 셀렉터)
                try:
                    desc_element = menu_element.find_element(By.CSS_SELECTOR, "div.TRxGt")
                    menu_info['description'] = desc_element.text.strip()
                except:
                    menu_info['description'] = "설명 없음"
                
                # 메뉴 이미지 (수정된 방식)
                try:
                    # 여러 이미지 셀렉터 시도
                    img_element = None
                    selectors = ["img.K0PDV", "img", "div.xPf1B img"]
                    
                    for selector in selectors:
                        try:
                            img_element = menu_element.find_element(By.CSS_SELECTOR, selector)
                            break
                        except:
                            continue
                    
                    if img_element:
                        img_url = img_element.get_attribute("src")
                        if img_url:
                            # 이미지 파일명 생성
                            img_filename = f"{restaurant_name}_menu_item_{i+1}_{menu_info['name']}.jpg"
                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            
                            # 이미지 다운로드
                            downloaded_path = download_image(img_url, img_filename, images_dir)
                            menu_info['image_path'] = downloaded_path
                        else:
                            menu_info['image_path'] = None
                    else:
                        menu_info['image_path'] = None
                except:
                    menu_info['image_path'] = None
                
                menu_list.append(menu_info)
                print(f"메뉴 {i+1}: {menu_info['name']} - {menu_info['price']}")
                
            except Exception as e:
                print(f"메뉴 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"메뉴 정보 수집 실패: {e}")
    
    # 메뉴판 사진도 수집
    menu_photos = get_menu_photos(images_dir, restaurant_name)
    
    return menu_list, menu_photos

def find_and_click_review_tab():
    """리뷰 탭을 정확하게 찾아서 클릭"""
    try:
        print("리뷰 탭 찾는 중...")
        
        # 페이지 스크롤하여 탭이 보이도록 함
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        
        # 1단계: 모든 탭 요소들 찾기
        tab_selectors = [
            "a._tab-menu",
            "div.YYh8o a",
            "div[class*='tab'] a",
            "nav a",
            "ul li a"
        ]
        
        all_tabs = []
        for selector in tab_selectors:
            try:
                tabs = driver.find_elements(By.CSS_SELECTOR, selector)
                if tabs:
                    all_tabs = tabs
                    print(f"탭 요소들 찾음: {len(tabs)}개")
                    break
            except:
                continue
        
        if not all_tabs:
            print("탭 요소를 찾을 수 없습니다")
            return False
        
        # 2단계: 각 탭의 텍스트와 URL 확인하여 리뷰 탭 찾기
        review_tab = None
        for i, tab in enumerate(all_tabs):
            try:
                # 탭 텍스트 확인
                tab_text = tab.text.strip().lower()
                # href 속성 확인
                tab_href = tab.get_attribute("href") or ""
                # data-index 확인
                tab_index = tab.get_attribute("data-index")
                
                print(f"탭 {i+1}: 텍스트='{tab_text}', href='{tab_href}', data-index='{tab_index}'")
                
                # 리뷰 탭인지 판단 (여러 조건으로 확인)
                is_review_tab = (
                    "review" in tab_href.lower() or
                    "리뷰" in tab_text or
                    "review" in tab_text
                )
                
                if is_review_tab:
                    review_tab = tab
                    print(f"리뷰 탭 발견: {tab_text}")
                    break
                    
            except Exception as e:
                print(f"탭 {i+1} 확인 중 오류: {e}")
                continue
        
        # 3단계: 리뷰 탭 클릭
        if review_tab:
            try:
                # 요소가 보이도록 스크롤
                driver.execute_script("arguments[0].scrollIntoView(true);", review_tab)
                time.sleep(1)
                
                # JavaScript로 클릭 시도 (element click intercepted 해결)
                driver.execute_script("arguments[0].click();", review_tab)
                time.sleep(3)
                print("리뷰 탭 클릭 완료")
                
                # 리뷰 콘텐츠가 로드되었는지 확인
                review_content_selectors = [
                    "li.place_apply_pui",
                    "div.pui__vn15t2",
                    "[class*='review']"
                ]
                
                for selector in review_content_selectors:
                    try:
                        elements = driver.find_elements(By.CSS_SELECTOR, selector)
                        if elements:
                            print(f"리뷰 콘텐츠 확인됨: {len(elements)}개 요소")
                            return True
                    except:
                        continue
                
                print("리뷰 콘텐츠를 찾을 수 없지만 탭 클릭은 완료")
                return True
                
            except Exception as e:
                print(f"리뷰 탭 클릭 실패: {e}")
                return False
        else:
            print("리뷰 탭을 찾을 수 없습니다")
            return False
            
    except Exception as e:
        print(f"리뷰 탭 찾기 실패: {e}")
        return False

def click_more_review_button():
    """리뷰 더보기 버튼 클릭"""
    try:
        # 여러 더보기 버튼 셀렉터 시도
        more_button_selectors = [
            "a.fvwqf",
            "button.fvwqf", 
            "a[class*='more']",
            "button[class*='more']",
            "//a[contains(text(), '더보기')]",
            "//button[contains(text(), '더보기')]",
            "//a[contains(text(), '리뷰 더보기')]"
        ]
        
        for selector in more_button_selectors:
            try:
                if selector.startswith("//"):
                    more_button = driver.find_element(By.XPATH, selector)
                else:
                    more_button = driver.find_element(By.CSS_SELECTOR, selector)
                
                if more_button.is_displayed() and more_button.is_enabled():
                    # 요소가 보이도록 스크롤
                    driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                    time.sleep(1)
                    
                    # JavaScript로 클릭
                    driver.execute_script("arguments[0].click();", more_button)
                    time.sleep(2)
                    print("리뷰 더보기 버튼 클릭 완료")
                    return True
            except:
                continue
                
        print("리뷰 더보기 버튼이 없거나 이미 모든 리뷰가 표시됨")
        return False
        
    except Exception as e:
        print(f"리뷰 더보기 버튼 처리 실패: {e}")
        return False

def get_review_info(images_dir, restaurant_name, max_reviews=20):
    """리뷰 탭에서 리뷰 정보 수집 (간소화 버전)"""
    review_list = []
    
    try:
        # 리뷰 더보기 버튼 클릭
        click_more_review_button()
        
        # 리뷰 항목들 찾기 (수정된 셀렉터들 시도)
        review_selectors = [
            "li.place_apply_pui.EjjAW",
            "div.pui__vn15t2",
            "li[class*='review']",
            "div[class*='review']"
        ]
        
        review_elements = []
        for selector in review_selectors:
            try:
                review_elements = driver.find_elements(By.CSS_SELECTOR, selector)[:max_reviews]
                if review_elements:
                    print(f"리뷰 요소 찾음: {selector} - {len(review_elements)}개")
                    break
            except:
                continue
        
        if not review_elements:
            print("리뷰 요소를 찾을 수 없습니다.")
            return review_list
        
        for i, review_element in enumerate(review_elements):
            review_info = {}
            
            try:
                # 리뷰 텍스트 (여러 셀렉터 시도)
                text_selectors = [
                    "span.zPfVt", 
                    "div.pui__vn15t2 span",
                    "div.pui__vn15t2",
                    "span[class*='review']", 
                    "div[class*='content']",
                    "p[class*='review']",
                    "div.YeINN",
                    "span.YeINN",
                    ".place_review_text",
                    "[class*='text']"
                ]
                review_text = "리뷰 텍스트 없음"
                
                for text_selector in text_selectors:
                    try:
                        text_elements = review_element.find_elements(By.CSS_SELECTOR, text_selector)
                        for text_element in text_elements:
                            text_content = text_element.text.strip()
                            # 의미있는 텍스트가 있는지 확인 (길이 10자 이상)
                            if text_content and len(text_content) > 10:
                                review_text = text_content
                                break
                        if review_text != "리뷰 텍스트 없음":
                            break
                    except:
                        continue
                
                # 여전히 텍스트를 못찾았다면 전체 텍스트에서 추출 시도
                if review_text == "리뷰 텍스트 없음":
                    try:
                        full_text = review_element.text.strip()
                        # 긴 텍스트가 있다면 첫 번째 문장 추출
                        if len(full_text) > 20:
                            sentences = full_text.split('\n')
                            for sentence in sentences:
                                if len(sentence.strip()) > 10:
                                    review_text = sentence.strip()
                                    break
                    except:
                        pass
                
                review_info['text'] = review_text
                
                # 리뷰 날짜 (여러 패턴 시도)
                date_selectors = ["time", "span.pui__gfuUIT", "div.pui__QKE5Pr", "span[class*='date']"]
                review_date = "날짜 정보 없음"
                
                for date_selector in date_selectors:
                    try:
                        date_element = review_element.find_element(By.CSS_SELECTOR, date_selector)
                        date_text = date_element.text.strip()
                        
                        # 날짜 패턴 추출
                        date_match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2}|\d{1,2}\.\d{1,2}|\d+일전|\d+주전|\d+개월전|\d+년전)', date_text)
                        if date_match:
                            review_date = date_match.group(1)
                            break
                        elif date_text and len(date_text) > 0:
                            review_date = date_text
                            break
                    except:
                        continue
                
                review_info['date'] = review_date
                
                # 리뷰 이미지들
                review_images = []
                try:
                    # 리뷰 이미지 슬라이드 처리
                    img_elements = review_element.find_elements(By.CSS_SELECTOR, "img")
                    for j, img_element in enumerate(img_elements):
                        img_url = img_element.get_attribute("src")
                        if img_url and ("review" in img_url or "ldb-phinf" in img_url):  # 리뷰 이미지 필터링
                            img_filename = f"{restaurant_name}_review_{i+1}_{j+1}.jpg"
                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            
                            downloaded_path = download_image(img_url, img_filename, images_dir)
                            if downloaded_path:
                                review_images.append(downloaded_path)
                    
                    # 리뷰 이미지 슬라이드 버튼 클릭해서 더 많은 이미지 보기
                    try:
                        next_button = review_element.find_element(By.CSS_SELECTOR, "a.ZCqf_")
                        click_count = 0
                        max_clicks = 5  # 리뷰당 최대 5번 클릭
                        
                        while click_count < max_clicks:
                            if next_button.is_displayed() and next_button.is_enabled():
                                next_button.click()
                                time.sleep(1)
                                click_count += 1
                                
                                # 새로운 이미지들 찾기
                                new_img_elements = review_element.find_elements(By.CSS_SELECTOR, "img")
                                for k, new_img_element in enumerate(new_img_elements[len(img_elements):]):
                                    try:
                                        img_url = new_img_element.get_attribute("src")
                                        if img_url and ("review" in img_url or "ldb-phinf" in img_url):
                                            img_filename = f"{restaurant_name}_review_{i+1}_{len(img_elements)+k+1}.jpg"
                                            img_filename = "".join(c for c in img_filename if c.isalnum() or c in (' ', '-', '_')).rstrip()
                                            
                                            downloaded_path = download_image(img_url, img_filename, images_dir)
                                            if downloaded_path:
                                                review_images.append(downloaded_path)
                                    except:
                                        continue
                            else:
                                break
                    except:
                        pass
                        
                except Exception as e:
                    print(f"리뷰 {i+1} 이미지 수집 실패: {e}")
                
                review_info['images'] = review_images
                review_list.append(review_info)
                print(f"리뷰 {i+1}: {review_info['text'][:50]}..." if len(review_info['text']) > 50 else f"리뷰 {i+1}: {review_info['text']}")
                
            except Exception as e:
                print(f"리뷰 {i+1} 정보 수집 실패: {e}")
                continue
                
    except Exception as e:
        print(f"리뷰 정보 수집 실패: {e}")
        
    return review_list

def go_back_to_list():
    """목록으로 돌아가기"""
    try:
        # 메인으로 돌아가기
        driver.switch_to.default_content()
        
        # 검색 결과 iframe으로 다시 전환
        search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
        driver.switch_to.frame(search_iframe)
        print("목록으로 돌아가기 완료")
        return True
        
    except Exception as e:
        print(f"목록으로 돌아가기 실패: {e}")
        return False

try:
    # 1. 검색어 설정 및 접속
    keyword = "신림동 짜장면"
    encoded_keyword = urllib.parse.quote(keyword)
    URL = f"https://map.naver.com/p/search/{encoded_keyword}"
    
    print(f"네이버 지도 접속 중: {URL}")
    driver.get(URL)
    time.sleep(5)
    
    # 2. 디렉토리 생성
    base_dir, images_dir = create_directory(keyword)
    
    # 3. 검색 결과 iframe 접근
    print("검색 결과 iframe으로 전환 중...")
    search_iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe")))
    driver.switch_to.frame(search_iframe)
    
    # 4. 가게 리스트 수집
    restaurant_elements = get_restaurant_list()
    total_restaurants = len(restaurant_elements)
    print(f"총 {total_restaurants}개 가게 발견")
    
    # 5. 각 가게별로 세부 정보 수집 (처음 5개만 테스트)
    max_restaurants = min(5, total_restaurants)  # 테스트용으로 5개만
    
    for i in range(max_restaurants):
        print(f"\n========== 가게 {i+1}/{max_restaurants} ==========")
        
        try:
            # 가게 클릭
            if not click_restaurant(restaurant_elements[i], i+1):
                continue
            
            # 세부 정보 iframe으로 전환
            if not switch_to_detail_iframe():
                continue
            
            # 홈 정보 수집
            print("홈 정보 수집 중...")
            home_info = get_home_info()
            
            # 메뉴 정보 수집
            menu_info = []
            menu_photos = []
            if find_and_click_menu_tab():
                print("메뉴 정보 수집 중...")
                menu_info, menu_photos = get_menu_info(images_dir, home_info.get('name', f'restaurant_{i+1}'))
            
            # 리뷰 정보 수집
            review_info = []
            if find_and_click_review_tab():
                print("리뷰 정보 수집 중...")
                review_info = get_review_info(images_dir, home_info.get('name', f'restaurant_{i+1}'))
            
            # 전체 정보 결합
            restaurant_data = {
                'index': i + 1,
                'home_info': home_info,
                'menu_info': menu_info,
                'menu_photos': menu_photos,
                'review_info': review_info,
                'collected_at': datetime.now().isoformat()
            }
            
            all_restaurant_data.append(restaurant_data)
            print(f"가게 {i+1} 정보 수집 완료!")
            
            # 목록으로 돌아가기
            go_back_to_list()
            time.sleep(2)
            
        except Exception as e:
            print(f"가게 {i+1} 처리 중 오류: {e}")
            # 목록으로 돌아가기 시도
            go_back_to_list()
            continue
    
    # 6. 결과 저장
    print(f"\n총 {len(all_restaurant_data)}개 가게 상세 정보 수집 완료!")
    
    # JSON 파일 저장
    json_filename = os.path.join(base_dir, f"{keyword}_detailed_restaurants.json")
    with open(json_filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(all_restaurant_data, jsonfile, ensure_ascii=False, indent=2)
    
    # CSV 파일 저장 (기본 정보만)
    csv_filename = os.path.join(base_dir, f"{keyword}_restaurants_summary.csv")
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['index', 'name', 'category', 'address', 'phone', 'menu_count', 'menu_photos_count', 'review_count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for restaurant in all_restaurant_data:
            row = {
                'index': restaurant['index'],
                'name': restaurant['home_info'].get('name', ''),
                'category': restaurant['home_info'].get('category', ''),
                'address': restaurant['home_info'].get('address', ''),
                'phone': restaurant['home_info'].get('phone', ''),
                'menu_count': len(restaurant['menu_info']),
                'menu_photos_count': len(restaurant['menu_photos']),
                'review_count': len(restaurant['review_info'])
            }
            writer.writerow(row)
    
    print(f"\n파일 저장 완료:")
    print(f"- 상세 정보 (JSON): {json_filename}")
    print(f"- 요약 정보 (CSV): {csv_filename}")
    print(f"- 이미지 폴더: {images_dir}")

except Exception as e:
    print(f"전체 프로세스 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 드라이버 종료
    driver.quit()

네이버 지도 접속 중: https://map.naver.com/p/search/%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4
검색 결과 iframe으로 전환 중...
가게 리스트 수집 중...
스크롤 컨테이너 찾음
현재 로드된 가게 수: 10
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
현재 로드된 가게 수: 50
더 이상 로드할 가게가 없음
총 50개 가게 발견

========== 가게 1/5 ==========
가게 1 클릭 완료
세부 정보 iframe으로 전환 완료
홈 정보 수집 중...
가게명: 중화요리 팔공
분류: 중식당
주소: 서울 관악구 남부순환로 1680
메뉴 탭 찾는 중...
탭 요소들 찾음: 6개
탭 1: 텍스트='홈', href='https://pcmap.place.naver.com/restaurant/1775787333/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202507042122&locale=ko&svcName=map_pcv5&searchText=%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4', data-index='0'
탭 2: 텍스트='소식', href='https://pcmap.place.naver.com/restaurant/1775787333/feed?entry=bmp&from=map&fromPanelNum=2&timestamp=202507042122&locale=ko&svcName=map_pcv5&searchText=%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4', data-index='1'
탭 3: 텍스트='메뉴', href='https://pcmap.place.naver.com/restaurant/1775787333/menu?entry=bmp&from=map&fromPanelNum=2&times

메뉴 탭 클릭 완료
메뉴 콘텐츠 확인됨: 15개 요소
메뉴 정보 수집 중...
메뉴 더보기 버튼 클릭 완료
메뉴 1: 양꼬치 - 13,000
메뉴 2: 동파육덮밥 - 7,000
메뉴 3: 뀌바로우 - 12,000
메뉴 4: 어향가지 - 12,000
메뉴 5: 토마토계란볶음 - 10,000
메뉴 6: 지삼선 - 15,000
메뉴 7: 갈릭새우 - 12,000
메뉴 8: 유린기 - 12,000
메뉴 9: 마라향궈 - 15,000
메뉴 10: 볶음짜장면 - 7,000
메뉴 11: 중화비빔밥 - 7,000
메뉴 12: 볶음밥 - 7,000
메뉴 13: 우육탕면 - 7,000
메뉴 14: 제육볶음 - 11,000
메뉴 15: 크림새우 - 15,000
메뉴 16: 깐풍기 - 12,000
메뉴 17: 가지볶음 - 12,000
메뉴 18: 쟁반짜장 - 15,000
메뉴 19: 누롱지탕 - 15,000
메뉴 20: 마라탕 - 12,000
메뉴 21: 간풍새우 - 15,000
리뷰 탭 찾는 중...
탭 요소들 찾음: 6개
탭 1: 텍스트='홈', href='https://pcmap.place.naver.com/restaurant/37537647/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202507042123&locale=ko&svcName=map_pcv5&searchText=%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4', data-index='0'
탭 2: 텍스트='소식', href='https://pcmap.place.naver.com/restaurant/37537647/feed?entry=bmp&from=map&fromPanelNum=2&timestamp=202507042123&locale=ko&svcName=map_pcv5&searchText=%EC%8B%A0%EB%A6%BC%EB%8F%99%20%EC%A7%9C%EC%9E%A5%EB%A9%B4', data-index=

리뷰 탭 클릭 완료
리뷰 콘텐츠 확인됨: 9개 요소
리뷰 정보 수집 중...
리뷰 더보기 버튼 클릭 완료
리뷰 요소 찾음: li.place_apply_pui.EjjAW - 9개
리뷰 1: 매장 분위기가 조용하고 음식도 완전 맛있었어요! 자주 올거같아요!
리뷰 2: 2023년 11월 20일 월요일
리뷰 3: 정갈하고 소박하면서도 품위있는 예식을 할 수 있었어요
리뷰 4: 20년 전 뷔페음식 같았어요;;;;
리뷰 5: 약간 비싸요. 맛은 괜찮아요
리뷰 6: 리뷰 1,817사진 280
리뷰 7: 리뷰 5,294사진 770팔로워 82
리뷰 8: 리뷰 269사진 71
리뷰 9: 김지은젤네일손톱녀짱83
가게 5 정보 수집 완료!
목록으로 돌아가기 완료

총 5개 가게 상세 정보 수집 완료!

파일 저장 완료:
- 상세 정보 (JSON): 신림동 짜장면_restaurant_data\신림동 짜장면_detailed_restaurants.json
- 요약 정보 (CSV): 신림동 짜장면_restaurant_data\신림동 짜장면_restaurants_summary.csv
- 이미지 폴더: 신림동 짜장면_restaurant_data\images


In [27]:
with open('신림동 짜장면_restaurant_data/신림동 짜장면_detailed_restaurants.json', encoding='utf-8') as f:
    data = json.load(f)
print(data)

[{'index': 1, 'home_info': {'name': '중화요리 팔공', 'category': '중식당', 'address': '서울 관악구 남부순환로 1680', 'phone': '0507-1307-9815', 'hours': '영업시간 정보 없음'}, 'menu_info': [{'name': '짜장면', 'price': '10,000', 'description': '설명 없음', 'image_path': '신림동 짜장면_restaurant_data\\images\\중화요리 팔공_menu_item_1_짜장면jpg'}, {'name': '팔공해물짬뽕', 'price': '13,000', 'description': '설명 없음', 'image_path': '신림동 짜장면_restaurant_data\\images\\중화요리 팔공_menu_item_2_팔공해물짬뽕jpg'}, {'name': '옛날볶음밥', 'price': '12,000', 'description': '설명 없음', 'image_path': '신림동 짜장면_restaurant_data\\images\\중화요리 팔공_menu_item_3_옛날볶음밥jpg'}, {'name': '잡채밥', 'price': '12,000', 'description': '설명 없음', 'image_path': '신림동 짜장면_restaurant_data\\images\\중화요리 팔공_menu_item_4_잡채밥jpg'}, {'name': '유산슬밥', 'price': '18,000', 'description': '설명 없음', 'image_path': None}, {'name': '잡탕밥', 'price': '19,000', 'description': '설명 없음', 'image_path': None}, {'name': '생등심 탕수육 소', 'price': '26,000', 'description': '설명 없음', 'image_path': '신림동 짜장면_restaurant_data\\images\\중화요리 

In [29]:
import pandas as pd

df = pd.read_json('신림동 짜장면_restaurant_data/신림동 짜장면_detailed_restaurants.json')
df

,index,home_info,menu_info,menu_photos,review_info,collected_at
0,1,"{'name': '중화요리 팔공', 'category': '중식당', 'addres...","[{'name': '짜장면', 'price': '10,000', 'descripti...",[],[{'text': '평일 오전 11시 도착. 앞에 웨이팅이 7팀 정도 있었지만 오픈...,2025-07-04 21:22:49.163470
1,2,"{'name': '리옌 본점', 'category': '중식당', 'address'...","[{'name': '해 코스요리', 'price': '60,000', 'descri...",[],"[{'text': '룸 이용했는데 조용하고 아늑했어요. 음식이 전반적으로 깔끔하고,...",2025-07-04 21:23:16.176108
2,3,"{'name': '만 성 찬 팅', 'category': '중식당', 'addres...","[{'name': '양꼬치', 'price': '13,000', 'descripti...",[],[{'text': '술마시기 정말 좋은 곳. 음식들이 전반적으로 짠데 맛있다. 마...,2025-07-04 21:23:43.354482
3,4,"{'name': '친향중식당', 'category': '중식당', 'address'...","[{'name': '탕수육 소', 'price': '12,000', 'descrip...",[],[{'text': '먹다가 사진을 깜빡해서 후다닥..찍었어요 고추잡채랑 볶음밥이 기...,2025-07-04 21:24:09.267079
4,5,"{'name': '이라운지 서울대점', 'category': '중식당', 'addr...","[{'name': '샤오마이(3pcs)', 'price': '5,000', 'des...",[],[{'text': '매장 분위기가 조용하고 음식도 완전 맛있었어요! 자주 올거같아요...,2025-07-04 21:24:30.992679
